<h2> DataDay 2024 </h2>
<h3> Som príliš často na sociálnej sieti, čo hovoria dáta? </h3>
<h4> doc. Mgr. Jozef Kiseľák, PhD. </h4>

Svoje facebookove údaje si môžte stiahnuť na linku: https://www.facebook.com/dyi/?referrer=yfi_settings
Alebo použite súbor your_posts_1.json z GitHubu.


Najprv si načítame knižnice (moduly), ktoré budeme využívať.

Medzi najzákladnejšie patrí knižnica Pandas, ktorá slúži na manipuláciu s dátami a ich analýzu. Na ďalšie operácie s dátovými štruktúrami nám bude slúžiť aj knižnica NumPy.

Ďalšími podstatnými knižnicami, ktoré využijeme sú knižnice Seaborn a Matplotlib, ktoré nám poslúžia na vizualizáciu našich údajov.

<h4> Importy </h4>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import adfuller

<h4> Načítanie údajov a ich spracovanie </h4>

Knižnica Pandas obsahuje užitočnú dátovu štruktúru DataFrame, do ktorej si načítame naše údaje.

DataFrame, alebo dátový rámec je štruktúra, ktorá je určená najmä pre tabuľkové údaje a vďaka nej vieme efektívne pracovať aj s veľkými súbormi.

In [ ]:
df = pd.read_json('your_posts_1.json')
#event = pd.read_json('event_invitations.json')

Funkcia .head() nám poskytne náhľad do našich údajov. Bez dodatočných argumentov nám vypíše prvých 5 riadkov DataFrame-u. Ak by sme chceli vypísať viac, alebo menej riadkov, stačí požadovaný počet riadkov zadať do zátvoriek.

In [ ]:
df.head()
# df.head(3)

In [ ]:
print(f'Rozmery nášho DataFrame-u (počet riadkov, počet stĺpcov): {df.shape}')

In [ ]:
#Pre jednoduchosť si premenujeme stĺpec s časovou pečiatkou "timestamp" na názov "date".
df.rename(columns={'timestamp': 'date'}, inplace=True)
df

In [ ]:
#Niektoré zo stĺpcov v DataFrame nebudeme potrebovať, čiže ich môžme "zahodiť".
df = df.drop(['attachments', 'title', 'tags'], axis=1)
df

In [ ]:
#Overenie, či má dátum správny formát YYYY-MM-DD HH:MM:SS.
df['date'] = pd.to_datetime(df['date'])
df

In [ ]:
print(f'Rozmery nášho DataFrame-u (počet riadkov, počet stĺpcov), po vymazaní stĺpcov: {df.shape}')

Podobne ako u funkcie .head(), ktorá nám vypísala počiatočné riadky, nám funkcia .tail() vypíše koncové riadky.

In [ ]:
df.tail()
# df.tail(3)

In [ ]:
#Nastavíme si v DataFrame ako hlavný index dátum.
df.set_index('date',inplace=True)

V nasledujúcej bunke sa naše údaje - teda interakcie na sociálnej sieti, rozdelia do skupiniek po mesiacoch v jednotlivých rokoch, v ktorých sa odohrali. Stĺpec "date" teraz nebude obsahovať informácie vo formáte YYYY-MM-DD HH:MM:SS, ale už len YYYY-MM-01, keďže sme nerozlišovali deň, v ktorých boli dané posty vytvorené.

Funkcia .size() nám uloží do stĺpca "data" počet príspevkov, ktoré boli v danom mesiaci daného roka vytvorené.

In [ ]:
post_counts=df.resample('MS').size()  #For 'MS' the dates of the groups are always the first of the month, for 'M' the last day.
#post_means=post_counts.resample('YS').mean(numeric_only=True)

Ukážka novozískaných údajov:

In [ ]:
post_counts

V ďalšom kroku si vyberieme nejaký počiatočný a koncový dátum, pre ktoré si z DataFrame-u o počte príspevkov za jednotlivé mesiace zoberieme podmnožinu údajov pre vizualizáciu.

In [ ]:
start_date = '2021-11-15'
end_date   = '2022-11-18'
#Nasledujúci príkaz šikovne zoberie len tie riadky, ktoré budú v nami zadanom rozsahu dátumov.
df2 = df.query('date >= @start_date and date <= @end_date')
df2

<h4> Vizualizácia </h4>

In [ ]:
#Nastavenie predvolenej veľkosti grafu a veľkosti písma.
sns.set(rc={'figure.figsize':(10,5)})
sns.set(font_scale=1)

In [ ]:
#Na os x budeme chcieť jednotlivé dátumy - vložínme si teda tento stĺpec do premennej x_labels.
x_labels = post_counts.index
x_labels

In [ ]:
fig, ax = plt.subplots()
ax.bar(x_labels,post_counts,color = "black",edgecolor = "black", linewidth = 2)
plt.show()

In [ ]:
plt.plot(post_counts, marker='o')
plt.show()

In [ ]:
ax = post_counts.plot(x='Date', y='Number of posts')
plt.show()

In [ ]:
xcoords = ['2008-01-01', '2009-01-01', '2010-01-01', '2011-01-01', '2012-01-01', '2013-01-01', '2014-01-01', '2015-01-01', '2016-01-01','2017-01-01', '2018-01-01', '2019-01-01', '2020-01-01', '2021-01-01', '2022-01-01', '2023-01-01']
for xc in xcoords:
    plt.axvline(x=xc, color='black', linestyle='--')
# plt.scatter(x_labels, post_counts)
# post_freq=post_counts/np.sum(post_counts)  #frekvencie
# plt.plot(post_freq, marker='o')

In [ ]:
post_counts.hist()
plt.show()

In [ ]:
plt.hist(post_counts, bins=20, linewidth=0.5, edgecolor="white")
plt.show()

In [ ]:
post_counts.plot(kind='kde')
plt.show()

In [ ]:
#More points tighter in to the diagonal line suggests a stronger relationship and more spread from the line suggests a weaker relationship.
#A ball in the middle or a spread across the plot suggests a weak or no relationship.
pd.plotting.lag_plot(post_counts, lag=1)
plt.show()

In [ ]:
plot_acf(post_counts)

In [ ]:
split = round(len(post_counts) / 2)
X1, X2 = post_counts[0:split], post_counts[split:]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()
print('mean1=%f, mean2=%f' % (mean1, mean2))
print('variance1=%f, variance2=%f' % (var1, var2))

In [ ]:
result = adfuller(post_counts)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('used lag: %f' % result[2])
print('num. of obs.: %f' % result[3])
print('Critical Values:')
for key, value in result[4].items():
 print('\t%s: %.3f' % (key, value))

print('The maximized information criterion if autolag is not None: %f' % result[5])

<h2> Zdroje: </h2>

Pôvodný Jupyter notebook FCBOOK.ipynb: https://colab.research.google.com/drive/1b355atUOd8xm6QbwfP6jkVQAmJoewIhZ